# Handwritten digits, SVD classification, and randomness
_What does it mean to be random?_

The key parameter of our SVD-based handwritten digit classifier is $k$. The projection matrices we use during classification are computed using the first $k$ singular vectors.

Recall that when $k=256$ (the full dimension of the data), each projection matrix is (at least theoretically) $UU^\top = I$. This means all our errors should be equal to zero, which of course doesn't help us classify digits (in fact, the algorithm performs worst when $k=256$).

But in class you may have noticed that even this worst-case performance seemed high. You might have seen over 70% accuracy even with $k=256$! What's going on here? Shouldn't the classification be random? Shouldn't the accuracy be closer to 10%?

Clearly there is some non-randomness in our data or our algorithm. Let's look for it.

_This notebook contains Python code, but don't worry if you aren't familiar with Python. All you need to do is run each cell and play with the output._

## Getting started
Run the following cell to load some code and our digit data.

In [ ]:
from scipy.io import loadmat
import ipywidgets as ipw
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# contains classify_image and related fcns:
from eigenimages import *

# load data
trn = loadmat("TRAIN_DIGITS.mat")["TRAIN_DIGITS"]
testdata = loadmat("TEST_DIGITS.mat")
tst = testdata["TEST_DIGITS"]
labels = testdata["TEST_DIGIT_LABELS"]

## Distribution of test data

One potential source of non-randomness lies in our data. What if we have different numbers of various digits? Let's look at the distribution.

In [ ]:
# compute the number of occurrences of each digit
digit_counts = np.zeros(10)
for i in range(10):
    digit_counts[i] = sum(labels == i)

# plot
fig = figure(figsize=(6,4))
xl = (-0.5,9.5)
yl = (100,400)
markerline, stemlines, baseline = stem(range(10), digit_counts, linefmt='k-', markerfmt='ko')
setp(stemlines, alpha=0.3)
title('Test Digit Distribution')
xlabel('Digit')
ylabel('Number of test digits')
xlim(xl)
ylim(yl)
xticks(range(10))
twinx()
ylabel('Fraction of test digits')
xlim(xl)
ylim(yl/sum(digit_counts))
yticks
grid()

Okay, so the digit "0" is over-represented among our test digit dataset. By contrast, only 7.5% of the test digits are "7". This might help explain why our algorithm appears to have non-random behavior when $k=256$.

**Why would this dataset have more zeros and ones than fives and sevens?*

One possible explanation is that real datasets (like the subset of [MNIST][1] we are using) actually exhibit this behavior. This phenomenon is referred to as [Benford's Law][2].

Now let's look at the distribution of our computer's "random" predictions when $k=256$.

[1]: https://en.wikipedia.org/wiki/MNIST_database
[2]: https://en.wikipedia.org/wiki/Benford%27s_law

## Distribution of predictions

Another potential source of non-randomness is the prediction algorithm itself. Rounding errors may cause our projection matrices to be slightly different from the identity matrix, so our classification errors may not be exactly zero. Even if the errors were all identical, the computer may not "roll a fair die" when confronted with multiple equal values -- it may choose the last in the set of identical values, for instance.

Let's plot the distribution of predictions for $k=256$.

In [ ]:
# classify all digits
k = 256
predictions = classify_image(tst,trn,k)

# compute the number of occurrences of each digit prediction
prediction_counts = np.zeros(10)
for i in range(10):
    prediction_counts[i] = sum(predictions == i)

# plot 
fig = figure(figsize=(6,4))
xl = (-0.5,9.5)
yl = (0,800)
markerline, stemlines, baseline = stem(range(10), prediction_counts, 
                                       linefmt='k-', markerfmt='ko')
setp(stemlines, alpha=0.3)
title('Prediction Distribution')
xlabel('Digit')
ylabel('Number of predicted digits')
# xlim(xl)
# ylim(yl)
xticks(range(10))
twinx()
ylabel('Fraction of predicted digits')
xlim(xl)
ylim(yl/sum(prediction_counts))
grid()

## Interactive: overlay of both distributions

Run the following cell to generate an interactive overlay of the test digit and prediction distributions.

In [ ]:
# plot
xl = np.array([-0.5,9.5])
yl = np.array([0,800])
fig = figure(figsize=(6,4))
ax_l = fig.add_subplot(111)
ax_l.set_xlim(xl)
ax_l.set_ylim(yl)
ax_l.set_xticks(range(10))
ax_l.set_xlabel('Digit')
ax_l.set_ylabel('Number of occurrences')
ax_l.xaxis.grid()

ax_r = ax_l.twinx()
ax_r.set_ylim(yl/np.float(size(labels)))
ax_r.set_ylabel('Fraction of all digits')

vals = ax_r.get_yticks()
ax_r.set_yticklabels(['{:3.0f}%'.format(x*100) for x in vals])

m1, s1, b1 = ax_l.stem(arange(10), digit_counts, 
                  linefmt='k-', markerfmt='bo')
setp(s1, alpha=0)
setp(b1,lw=0)

prediction_counts = np.zeros(10)
m2, s2, b2 = ax_l.stem(arange(10), prediction_counts, 
                  linefmt='k-', markerfmt='ro')
setp(s2, alpha=0)
setp(b2,lw=0)

ax_l.legend(['Test Data Labels','Prediction Labels'], fontsize=10)

def on_change(k):
    # classify all digits
    predictions = classify_image(tst,trn,k)

    # compute the number of occurrences of each digit prediction
    prediction_counts = np.zeros(10)
    for i in range(10):
        prediction_counts[i] = sum(predictions == i)

    m2.set_ydata(prediction_counts)
    ax_r.set_title('Prediction Distribution (accuracy = {:3.0f}%)'.format(100*sum(predictions == labels)/size(labels)))
    return fig

ipw.interact(on_change, k=ipw.IntSlider(min=1,max=256,step=20))

Neither of the two distributions is flat, and the final accuracy of the algorithm depends on both of them.

**Set $k=220$ by clicking on the number next to the slider and typing it in. Can you explain why the accuracy is so high by looking at the plot?**

**Now set $k=256$ and look at the distribution of prediction labels.**

The algorithm predicts 3, 4, or 5 90% of the time! Not very random...

## Bonus: which pairs of digits are difficult to distinguish?

Another way to look at our classification algorithm is to compute a 10-by-10 matrix $X$ where $X[i,j]$ element is the fraction of digit $j$ classified as digit $i$ by the algorithm. In other words, if $X[3,4] = 0.09$, the algorithm classified 9% of the "4"s as "3"s.

**What should each column of $X$ sum to?**

Now run the following cell to generate $X$ for any value of $k$ you choose. Note the "Color adjust" slider, which determines the black threshold. You may need to drag this slider down so you can see the lighter-colored squares.

In [ ]:
def prediction_by_digit(predictions, labels):
    """
    Return a matrix whose [i,j] element equals the fraction of test
    digits with label j are predicted to be digit i. (Cols sum to 1.)
    """
    prediction_spread = np.zeros((10,10))
    for i in range(10):
        for j in range(10):
            prediction_spread[i,j] = sum(predictions[labels == j] == i)/np.float(sum(labels == j))
    return prediction_spread

fig = figure(figsize=(6,6))
ax = fig.add_subplot(111)
set_cmap('gray_r')
im = ax.imshow(np.zeros((10,10)), interpolation='nearest')
im.set_clim((0,0.6))
colorbar(im)

xlabel('Test digit')
ylabel('Predicted digit')
xticks(range(10))
yticks(range(10))

labels = labels.squeeze()
def on_change(val,cmax):
    predictions = classify_image(tst,trn,val)
    prediction_spread = prediction_by_digit(predictions, labels)

    im.set_data(prediction_spread)
    fig.canvas.draw()
    ax.set_title('Prediction accuracy: ' + str(round(100*sum(predictions == labels)/size(labels),0)) + '%')
    im.set_clim((0,cmax))
    
    return fig

ipw.interact(on_change,
             val=ipw.IntSlider(
        min=1,
        max=256,
        step=10,
        description="k:",
        width=200),
             cmax=ipw.FloatSlider(
        description='Color adjust',
        min=0.2,
        max=1.0,
        step=0.1,
        value=1.0)
            )

**What would this plot look like ideally?**

**What value of $k$ comes closest to this ideal?**

**Why is there such a big difference in the plot between $k=255$ and $k=256$?**